In [14]:
# Call neccesary liraries
from Bio import PDB
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import numpy as np
import torch
import pydssp
import biolib

# Get PDB data
pdb_code = "2RH1"
pdb_file = PDB.PDBList()
pdb_file.retrieve_pdb_file(pdb_code, file_format="pdb")

Structure exists: 'C:\Users\Student\OneDrive - Aston University\Documents\Biology\Project\Project_automation\Python\rh\pdb2rh1.ent' 


'C:\\Users\\Student\\OneDrive - Aston University\\Documents\\Biology\\Project\\Project_automation\\Python\\rh\\pdb2rh1.ent'

In [15]:
# Create a dictionary for amino acid conversion 3 letter to one
three_to_one = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

In [16]:
# Parse the file
parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure(pdb_code, f"rh\\pdb2rh1.pdb")

# Select the first structure in the file
model = structure[0]

# Select the A chain
chain = model['A']


In [4]:
# Retrieve coordinates
coordinates = []
fasta_seq = []
for residue in chain:
    if PDB.is_aa(residue):
        res_name = residue.get_resname()
        fasta_seq.append(three_to_one.get(res_name))
        for atom in residue:
            if atom.get_name() in ['N', 'CA', 'C', 'O']:
                coordinates.append(atom.coord)

In [29]:
print(len(fasta_seq))
print(len(coordinates))

442
1768


In [5]:
# Prepare eliments of pytorch tensor
L = sum(1 for residue in chain if PDB.is_aa(residue))
atoms = 4
xyz = 3

# Make coordinates into array first to increase efficiency
coord_array = np.array(coordinates)

# Create tensor
coord_tensor = torch.tensor(coord_array, dtype=torch.float32).reshape([L, atoms, xyz])

In [6]:
#dssp_hbond_matrix = pydssp.get_hbond_map(coord_tensor) > 0.5

# Use pydssp to get secondary structure
ss_data = pydssp.assign(coord_tensor, out_type='c3')
print(ss_data)

['-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' 'H' 'H' 'H'
 '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-' '-' '-'
 '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-'
 '-' '-' 'E' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-' '-' '-'
 '-' '-' '-' '-' 'E' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-' '-' 'E' '-'
 'E' '-' 'E' 'E' '-' '-' '-' '-' '-' 'E' 'E' 'E' 'E' '-' '-' 'E' 'E' '-'
 'E' '-' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H

In [7]:
ss_data_string = "".join(ss_data)
print(ss_data_string)

--HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH-HHH--HHHHHHHHHHHHHHHHHHH-HHHHHHHHHH-----HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH----------HHHHHHHHHHHHHHHHHHHHHHHH----E---HHHHHHHH--------E-HHHHHHHHHHH-HHHHHHHHHHHHHHHHHHHHH--HHHHHHHH---E-E-EE-----EEEE--EE-E----HHHHHHHHHHHH------E--HHHHHHHHHHHHHHHHHHHHH-HHHHHHHHH--HHHHHHHHHHHHHHHHHHHH--HHHHHHHH---HHHHHHHH---HHHHH-HHHHHHHHHHHHH---HHH----HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH------HHHHHHHHHHHHHH---HHHHHHH-HHHHHHHHHH---
